In [ ]:
import os
import json
import asyncio
from typing import Dict, List, Optional, Union, Any
from datetime import datetime
from collections import deque
from enum import Enum
from pydantic import BaseModel, Field
import logging
import hashlib
from pathlib import Path
import ctransformers
from fastapi import FastAPI, WebSocket, HTTPException, BackgroundTasks
import uvicorn

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("agent_system.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("agent_system")

# ---- Models and Enums ----

class AgentType(str, Enum):
    PROJECT_MANAGER = "project_manager"
    PROGRAMMER = "programmer"

class ExpertiseArea(str, Enum):
    PYTHON = "python"
    JAVASCRIPT = "javascript"
    DOCKER = "docker"
    KUBERNETES = "kubernetes"
    DATABASE = "database"
    NETWORKING = "networking"
    WEB = "web"
    TESTING = "testing"
    DEVOPS = "devops"

class TaskStatus(str, Enum):
    PENDING = "pending"
    IN_PROGRESS = "in_progress"
    COMPLETED = "completed"
    BLOCKED = "blocked"
    FAILED = "failed"

class MessageType(str, Enum):
    TASK_ASSIGNMENT = "task_assignment"
    STATUS_UPDATE = "status_update"
    TECHNICAL_QUERY = "technical_query"
    FEEDBACK = "feedback"
    COMMAND_RESULT = "command_result"
    LEARNING = "learning"

class Thought(BaseModel):
    id: str
    timestamp: datetime = Field(default_factory=datetime.now)
    category: str
    content: str
    confidence: float = 0.5
    references: List[str] = []
    context_hash: str = ""  # Hash of the context that generated this thought
    
    def generate_id(self) -> str:
        """Generate a unique ID for the thought based on content and timestamp"""
        return hashlib.md5(f"{self.content}_{self.timestamp}".encode()).hexdigest()

class CommandExecutionRecord(BaseModel):
    command: str
    stdout: str
    stderr: str
    syslog: Optional[str] = None
    exit_code: int
    execution_time: float
    system_state_before: Dict[str, Any] = {}
    system_state_after: Dict[str, Any] = {}
    interaction_history: List[Dict[str, str]] = []
    agent_thoughts: List[Thought] = []
    success: bool = False
    annotations: Dict[str, Any] = {}

class Task(BaseModel):
    id: str
    title: str
    description: str
    status: TaskStatus = TaskStatus.PENDING
    assigned_to: Optional[str] = None
    due_date: Optional[datetime] = None
    required_expertise: List[ExpertiseArea] = []
    dependencies: List[str] = []
    priority: int = 1
    estimated_hours: float = 1.0
    actual_hours: float = 0.0
    progress: float = 0.0
    commands_executed: List[str] = []
    command_records: List[CommandExecutionRecord] = []
    subtasks: List[str] = []
    created_at: datetime = Field(default_factory=datetime.now)
    updated_at: datetime = Field(default_factory=datetime.now)

class Message(BaseModel):
    id: str = Field(default_factory=lambda: hashlib.md5(str(datetime.now().timestamp()).encode()).hexdigest())
    sender: str
    receiver: str
    message_type: MessageType
    content: Dict[str, Any]
    timestamp: datetime = Field(default_factory=datetime.now)
    read: bool = False

class MemoryEntry(BaseModel):
    timestamp: datetime = Field(default_factory=datetime.now)
    category: str
    content: Any
    importance: float = 1.0
    references: List[str] = []
    last_accessed: datetime = Field(default_factory=datetime.now)
    access_count: int = 0

class ThoughtMemory(BaseModel):
    capacity: int = 1000
    thoughts: List[Thought] = []
    thought_clusters: Dict[str, List[str]] = {}  # category -> thought_ids
    pattern_index: Dict[str, List[str]] = {}  # pattern -> thought_ids
    important_thoughts: List[str] = []  # Thought IDs deemed important
    recent_thoughts: deque = Field(default_factory=lambda: deque(maxlen=100))
    
    def add_thought(self, thought: Thought) -> None:
        # Ensure thought has an ID
        if not thought.id:
            thought.id = thought.generate_id()
            
        # Add to main thoughts list
        self.thoughts.append(thought)
        
        # Update indexes
        category = thought.category
        if category not in self.thought_clusters:
            self.thought_clusters[category] = []
        self.thought_clusters[category].append(thought.id)
        
        # Add to recent thoughts
        self.recent_thoughts.append(thought.id)
        
        # If we're over capacity, remove least important thoughts
        if len(self.thoughts) > self.capacity:
            self._prune_thoughts()
    
    def retrieve_relevant_thoughts(self, context: str, categories: List[str] = None, limit: int = 5) -> List[Thought]:
        # Basic implementation - would be enhanced with embeddings in a real system
        relevant_thoughts = []
        context_words = set(context.lower().split())
        
        for thought in self.thoughts:
            if categories and thought.category not in categories:
                continue
                
            thought_words = set(thought.content.lower().split())
            overlap = len(context_words.intersection(thought_words))
            if overlap > 3:  # Simple relevance threshold
                relevant_thoughts.append((thought, overlap))
        
        # Sort by relevance and return top results
        relevant_thoughts.sort(key=lambda x: x[1], reverse=True)
        return [t[0] for t in relevant_thoughts[:limit]]
    
    def _prune_thoughts(self) -> None:
        # Remove least important thoughts while preserving important ones
        thoughts_with_scores = []
        important_ids = set(self.important_thoughts)
        recent_ids = set(self.recent_thoughts)
        
        for thought in self.thoughts:
            # Calculate importance score based on confidence, recency, and marked importance
            score = thought.confidence
            if thought.id in important_ids:
                score += 10
            if thought.id in recent_ids:
                score += 5
                
            thoughts_with_scores.append((thought, score))
        
        # Sort by score ascending (least important first)
        thoughts_with_scores.sort(key=lambda x: x[1])
        
        # Remove enough to get back to capacity
        excess = len(self.thoughts) - self.capacity
        thoughts_to_remove = thoughts_with_scores[:excess]
        
        # Update data structures
        for thought, _ in thoughts_to_remove:
            self.thoughts.remove(thought)
            if thought.id in self.thought_clusters.get(thought.category, []):
                self.thought_clusters[thought.category].remove(thought.id)

class LearningMemory(BaseModel):
    experiences: List[CommandExecutionRecord] = []
    pattern_frequency: Dict[str, int] = {}
    success_patterns: Dict[str, int] = {}
    error_patterns: Dict[str, int] = {}
    learning_metrics: Dict[str, float] = {}
    max_size: int = 1000
    
    def add_record(self, record: CommandExecutionRecord) -> None:
        """Add a new command execution record and update patterns"""
        self.experiences.append(record)
        
        # Extract patterns
        patterns = self._extract_patterns(record)
        
        # Update pattern frequencies
        for pattern in patterns:
            self.pattern_frequency[pattern] = self.pattern_frequency.get(pattern, 0) + 1
            
            # Update success or error patterns
            if record.success:
                self.success_patterns[pattern] = self.success_patterns.get(pattern, 0) + 1
            else:
                self.error_patterns[pattern] = self.error_patterns.get(pattern, 0) + 1
                
        # Prune if needed
        if len(self.experiences) > self.max_size:
            self.experiences.pop(0)  # Remove oldest
    
    def _extract_patterns(self, record: CommandExecutionRecord) -> List[str]:
        """Extract patterns from command execution record"""
        patterns = []
        
        # Command patterns (simplified)
        cmd_parts = record.command.split()
        if len(cmd_parts) > 0:
            patterns.append(f"CMD:{cmd_parts[0]}")
            
        # Error patterns
        if record.stderr and len(record.stderr) > 0:
            error_lines = record.stderr.split('\n')
            for line in error_lines:
                if "error" in line.lower() or "exception" in line.lower():
                    patterns.append(f"ERR:{line[:50]}")  # First 50 chars of error
        
        # Exit code patterns
        patterns.append(f"EXIT:{record.exit_code}")
        
        # More complex patterns would be added here in a full implementation
        
        return patterns
    
    def get_relevant_experiences(self, command: str, limit: int = 5) -> List[CommandExecutionRecord]:
        """Retrieve relevant past experiences for a command"""
        cmd_parts = command.split()
        main_command = cmd_parts[0] if cmd_parts else ""
        
        # Find records with the same command
        similar_records = [
            rec for rec in self.experiences 
            if rec.command.split()[0] == main_command
        ]
        
        # Sort by recency (newer first)
        similar_records.sort(key=lambda x: x.execution_time, reverse=True)
        
        return similar_records[:limit]

# Base Agent Class
class Agent(BaseModel):
    id: str
    name: str
    agent_type: AgentType
    expertise: List[ExpertiseArea] = []
    model_path: str
    manual_paths: List[str] = []
    memory: List[MemoryEntry] = []
    thought_memory: ThoughtMemory = Field(default_factory=ThoughtMemory)
    learning_memory: LearningMemory = Field(default_factory=LearningMemory)
    inbox: List[Message] = []
    outbox: List[Message] = []
    max_memory_size: int = 1000
    llm: Any = None
    
    class Config:
        arbitrary_types_allowed = True
    
    def initialize(self):
        """Initialize the agent, including loading the LLM"""
        self.llm = self._load_llm()
        self._load_manuals()
        logger.info(f"Agent {self.name} ({self.id}) initialized with type {self.agent_type}")
    
    def _load_llm(self):
        """Load the LLM using ctransformers"""
        logger.info(f"Loading LLM from {self.model_path}")
        try:
            # For Phi-3 Mini in GGUF format
            model = ctransformers.AutoModelForCausalLM.from_pretrained(
                self.model_path,
                model_type="phi",
                gpu_layers=0  # Set higher for GPU acceleration
            )
            return model
        except Exception as e:
            logger.error(f"Error loading LLM: {e}")
            raise
    
    def _load_manuals(self):
        """Load agent manuals into memory"""
        for manual_path in self.manual_paths:
            try:
                with open(manual_path, 'r') as f:
                    content = f.read()
                    
                manual_name = Path(manual_path).stem
                
                # Store in memory
                self.add_to_memory(
                    category="manual",
                    content=content,
                    importance=2.0,
                    references=[manual_path]
                )
                logger.info(f"Loaded manual: {manual_name}")
            except Exception as e:
                logger.error(f"Error loading manual {manual_path}: {e}")
    
    def add_to_memory(self, category: str, content: Any, importance: float = 1.0, references: List[str] = []):
        """Add a new entry to agent memory"""
        entry = MemoryEntry(
            category=category,
            content=content,
            importance=importance,
            references=references
        )
        
        self.memory.append(entry)
        
        # Prune memory if needed
        if len(self.memory) > self.max_memory_size:
            self._prune_memory()
    
    def _prune_memory(self):
        """Remove least important memories to stay within size limit"""
        # Sort by importance and last_accessed (keep important and recent memories)
        self.memory.sort(key=lambda x: (x.importance, x.last_accessed.timestamp()))
        
        # Remove oldest, least important memories
        excess = len(self.memory) - self.max_memory_size
        self.memory = self.memory[excess:]
    
    def retrieve_from_memory(self, category: str, query: str = None, limit: int = 5):
        """Retrieve entries from memory matching category and optionally containing query"""
        results = [entry for entry in self.memory if entry.category == category]
        
        if query:
            # Very simple matching - would use embeddings in production
            results = [
                entry for entry in results 
                if query.lower() in str(entry.content).lower()
            ]
        
        # Update access stats
        for entry in results:
            entry.last_accessed = datetime.now()
            entry.access_count += 1
        
        # Sort by importance and return top results
        results.sort(key=lambda x: x.importance, reverse=True)
        return results[:limit]
    
    def receive_message(self, message: Message):
        """Receive a message into inbox"""
        self.inbox.append(message)
        logger.info(f"Agent {self.name} received message from {message.sender}: {message.message_type}")
    
    def send_message(self, to: str, message_type: MessageType, content: Dict[str, Any]):
        """Create and send a message"""
        message = Message(
            sender=self.id,
            receiver=to,
            message_type=message_type,
            content=content
        )
        self.outbox.append(message)
        logger.info(f"Agent {self.name} sent message to {to}: {message_type}")
        return message
    
    def process_messages(self):
        """Process all messages in inbox"""
        messages_to_process = self.inbox.copy()
        self.inbox = []
        
        for message in messages_to_process:
            self._process_message(message)
            # Mark as read
            message.read = True
    
    def _process_message(self, message: Message):
        """Process a single message - to be implemented by subclasses"""
        pass
    
    def generate_thought(self, category: str, context: str, confidence: float = 0.5) -> Thought:
        """Generate an internal thought based on the current context"""
        # Create context hash
        context_hash = hashlib.md5(context.encode()).hexdigest()
        
        # Build prompt for the LLM to generate a thought
        prompt = self._build_thought_prompt(category, context)
        
        # Generate thought content using the LLM
        thought_content = self._generate_with_llm(prompt, max_tokens=100)
        
        # Create the thought
        thought = Thought(
            id="",  # Will be generated on add
            category=category,
            content=thought_content,
            confidence=confidence,
            context_hash=context_hash
        )
        
        # Add to thought memory
        self.thought_memory.add_thought(thought)
        
        return thought
    
    def _build_thought_prompt(self, category: str, context: str) -> str:
        """Build a prompt for generating a thought of a specific category"""
        # Retrieve relevant past thoughts for context
        relevant_thoughts = self.thought_memory.retrieve_relevant_thoughts(
            context=context, 
            categories=[category], 
            limit=3
        )
        
        relevant_thought_text = "\n".join([
            f"- {thought.content}" for thought in relevant_thoughts
        ])
        
        prompts = {
            "observation": f"""
Based on the following context, generate a concise observation about what you notice:
CONTEXT:
{context}

PAST OBSERVATIONS:
{relevant_thought_text}

OBSERVATION:
""",
            "analysis": f"""
Analyze the following context and extract key insights:
CONTEXT:
{context}

PAST ANALYSIS:
{relevant_thought_text}

ANALYSIS:
""",
            "decision": f"""
Based on the following context, what decision would you make:
CONTEXT:
{context}

PAST DECISIONS:
{relevant_thought_text}

DECISION:
""",
            "learning": f"""
What can be learned from the following context:
CONTEXT:
{context}

PAST LEARNINGS:
{relevant_thought_text}

LEARNING:
"""
        }
        
        return prompts.get(category, f"Generate a {category} thought about: {context}")
    
    def _generate_with_llm(self, prompt: str, max_tokens: int = 512) -> str:
        """Generate text using the loaded LLM"""
        try:
            # Generate text with the model
            response = self.llm(
                prompt,
                max_new_tokens=max_tokens,
                temperature=0.7,
                top_p=0.95,
                repetition_penalty=1.1
            )
            
            # Clean and return the response
            return response.strip()
        except Exception as e:
            logger.error(f"Error generating with LLM: {e}")
            return f"Error generating thought: {str(e)}"
    
    async def step(self):
        """Process one step of the agent's loop - to be implemented by subclasses"""
        pass
    
    def summarize_context(self, context: str, max_length: int = 200) -> str:
        """Summarize a long context to keep it lightweight"""
        if len(context) <= max_length:
            return context
            
        prompt = f"Summarize the following text in a concise way:\n\n{context[:1000]}..."
        summary = self._generate_with_llm(prompt, max_tokens=200)
        
        return summary

# Project Manager Agent
class ProjectManagerAgent(Agent):
    current_tasks: List[Task] = []
    completed_tasks: List[Task] = []
    team: Dict[str, str] = {}  # programmer_id -> expertise
    
    async def step(self):
        """Run one step of the project manager's loop"""
        # Process incoming messages
        self.process_messages()
        
        # Check for tasks that need assignment
        await self._assign_pending_tasks()
        
        # Check for status updates needed
        await self._check_task_statuses()
        
        # Generate learning insights and feedback
        await self._generate_insights()
    
    def _process_message(self, message: Message):
        """Process a single message"""
        if message.message_type == MessageType.STATUS_UPDATE:
            self._handle_status_update(message)
        elif message.message_type == MessageType.TECHNICAL_QUERY:
            self._handle_technical_query(message)
        elif message.message_type == MessageType.COMMAND_RESULT:
            self._handle_command_result(message)
        
        # Generate thoughts about the message
        context = f"Message from {message.sender} of type {message.message_type}: {json.dumps(message.content)}"
        self.generate_thought("observation", context)
    
    def _handle_status_update(self, message: Message):
        """Handle a status update from a programmer"""
        task_id = message.content.get("task_id")
        new_status = message.content.get("status")
        progress = message.content.get("progress", 0.0)
        
        # Update task status
        for task in self.current_tasks:
            if task.id == task_id:
                old_status = task.status
                task.status = TaskStatus(new_status)
                task.progress = progress
                task.updated_at = datetime.now()
                
                # Move to completed if done
                if task.status == TaskStatus.COMPLETED:
                    self.completed_tasks.append(task)
                    self.current_tasks.remove(task)
                
                # Generate thought about the status change
                context = f"Task {task.title} status changed from {old_status} to {new_status} with progress {progress}"
                self.generate_thought("analysis", context)
                
                # Send feedback
                self._send_feedback(task)
                break
    
    def _handle_technical_query(self, message: Message):
        """Handle a technical query from a programmer"""
        problem = message.content.get("problem", {})
        task_id = message.content.get("task_id")
        
        # Generate thought about the problem
        context = f"Technical query from {message.sender} regarding task {task_id}: {json.dumps(problem)}"
        analysis_thought = self.generate_thought("analysis", context)
        
        # Determine if this needs to be escalated to architect
        complexity = self._assess_problem_complexity(problem)
        
        if complexity > 0.7:  # High complexity
            # Forward to architect
            self.send_message(
                to="architect",  # Assuming architect has this ID
                message_type=MessageType.TECHNICAL_QUERY,
                content={
                    "original_sender": message.sender,
                    "problem": problem,
                    "task_id": task_id,
                    "pm_analysis": analysis_thought.content
                }
            )
            
            # Let programmer know it's been escalated
            self.send_message(
                to=message.sender,
                message_type=MessageType.FEEDBACK,
                content={
                    "type": "technical_query_response",
                    "message": "Your query has been escalated to the architect for expert assistance.",
                    "task_id": task_id
                }
            )
        else:
            # Try to handle it with the PM's knowledge
            solution = self._generate_solution(problem)
            
            self.send_message(
                to=message.sender,
                message_type=MessageType.FEEDBACK,
                content={
                    "type": "technical_query_response",
                    "message": solution,
                    "task_id": task_id
                }
            )
    
    def _handle_command_result(self, message: Message):
        """Handle command execution results"""
        record = CommandExecutionRecord(**message.content.get("record", {}))
        task_id = message.content.get("task_id")
        
        # Find the task
        for task in self.current_tasks:
            if task.id == task_id:
                # Add to task records
                task.commands_executed.append(record.command)
                task.command_records.append(record)
                break
        
        # Add to learning memory
        self.learning_memory.add_record(record)
        
        # Generate thoughts about the command execution
        context = f"Command execution: {record.command}\nSuccess: {record.success}\nExit code: {record.exit_code}"
        self.generate_thought("observation", context)
        
        if not record.success:
            # Generate analysis thought for failed command
            error_context = f"Command failed: {record.command}\nError: {record.stderr}\nExit code: {record.exit_code}"
            self.generate_thought("analysis", error_context)
    
    async def _assign_pending_tasks(self):
        """Assign pending tasks to programmers"""
        # Find unassigned tasks
        pending_tasks = [task for task in self.current_tasks if task.status == TaskStatus.PENDING and not task.assigned_to]
        
        for task in pending_tasks:
            # Find best programmer based on expertise
            best_programmer = self._find_best_programmer(task)
            
            if best_programmer:
                # Assign task
                task.assigned_to = best_programmer
                task.status = TaskStatus.IN_PROGRESS
                task.updated_at = datetime.now()
                
                # Send assignment message
                self.send_message(
                    to=best_programmer,
                    message_type=MessageType.TASK_ASSIGNMENT,
                    content={
                        "task": task.dict()
                    }
                )
                
                # Generate thought about assignment
                context = f"Assigned task {task.title} to {best_programmer} based on expertise match with {task.required_expertise}"
                self.generate_thought("decision", context)
    
    async def _check_task_statuses(self):
        """Check status of ongoing tasks"""
        in_progress_tasks = [task for task in self.current_tasks if task.status == TaskStatus.IN_PROGRESS]
        
        for task in in_progress_tasks:
            # Check if task is overdue
            if task.due_date and datetime.now() > task.due_date:
                # Send reminder
                self.send_message(
                    to=task.assigned_to,
                    message_type=MessageType.FEEDBACK,
                    content={
                        "type": "reminder",
                        "message": f"Task '{task.title}' is overdue. Please provide an update.",
                        "task_id": task.id
                    }
                )
    
    async def _generate_insights(self):
        """Generate learning insights from completed tasks"""
        if len(self.completed_tasks) > 0:
            # Analyze patterns from completed tasks
            successful_patterns = {}
            for task in self.completed_tasks:
                for record in task.command_records:
                    if record.success:
                        patterns = self.learning_memory._extract_patterns(record)
                        for pattern in patterns:
                            successful_patterns[pattern] = successful_patterns.get(pattern, 0) + 1
            
            # Generate insights
            top_patterns = sorted(successful_patterns.items(), key=lambda x: x[1], reverse=True)[:5]
            
            if top_patterns:
                patterns_text = "\n".join([f"- {pattern} (observed {count} times)" for pattern, count in top_patterns])
                context = f"Successful patterns from completed tasks:\n{patterns_text}"
                insight = self.generate_thought("learning", context)
                
                # Share insights with programmers
                for programmer_id in self.team:
                    self.send_message(
                        to=programmer_id,
                        message_type=MessageType.LEARNING,
                        content={
                            "insight": insight.content,
                            "patterns": dict(top_patterns)
                        }
                    )
    
    def _find_best_programmer(self, task: Task) -> Optional[str]:
        """Find the best programmer for a task based on expertise and current workload"""
        matches = []
        
        for programmer_id, expertise_areas in self.team.items():
            # Check if programmer's expertise matches task requirements
            expertise_match = any(exp in expertise_areas for exp in task.required_expertise)
            
            if expertise_match:
                # Count current tasks assigned to this programmer
                workload = sum(1 for t in self.current_tasks if t.assigned_to == programmer_id)
                matches.append((programmer_id, workload))
        
        if not matches:
            return None
            
        # Sort by workload (ascending)
        matches.sort(key=lambda x: x[1])
        
        return matches[0][0]
    
    def _send_feedback(self, task: Task):
        """Send feedback on a task to the assigned programmer"""
        # Generate feedback based on task progress and quality
        if task.status == TaskStatus.COMPLETED:
            prompt = f"""
Task '{task.title}' has been completed.
Description: {task.description}
Commands executed: {', '.join(task.commands_executed[:5])}

Generate positive feedback and constructive improvement suggestions:
"""
        else:
            prompt = f"""
Task '{task.title}' is at {task.progress}% progress with status {task.status}.
Description: {task.description}
Commands executed: {', '.join(task.commands_executed[:5])}

Generate constructive feedback on work so far:
"""
        
        feedback = self._generate_with_llm(prompt)
        
        self.send_message(
            to=task.assigned_to,
            message_type=MessageType.FEEDBACK,
            content={
                "task_id": task.id,
                "feedback": feedback,
                "progress_assessment": task.progress
            }
        )
    
    def _assess_problem_complexity(self, problem: Dict) -> float:
        """Assess the complexity of a technical problem (0-1 scale)"""
        # This would be more sophisticated in a real implementation
        description = problem.get("description", "")
        context = problem.get("context", {})
        
        # Count indicators of complexity
        complexity_indicators = [
            "complex", "challenging", "difficult", "unknown", "error", 
            "exception", "failing", "segmentation fault", "core dump"
        ]
        
        count = sum(1 for word in complexity_indicators if word in description.lower())
        
        # Normalize to 0-1
        return min(1.0, count / 5)
    
    def _generate_solution(self, problem: Dict) -> str:
        """Generate a solution to a technical problem"""
        description = problem.get("description", "")
        context = problem.get("context", {})
        
        # Retrieve relevant memories
        relevant_memories = self.retrieve_from_memory("command_record", description, limit=3)
        
        # Format memory context
        memory_context = ""
        for memory in relevant_memories:
            cmd_record = memory.content
            memory_context += f"Command: {cmd_record.get('command')}\n"
            memory_context += f"Result: {cmd_record.get('success', False)}\n"
            if 'stderr' in cmd_record and cmd_record['stderr']:
                memory_context += f"Error: {cmd_record.get('stderr')}\n"
            memory_context += "\n"
        
        prompt = f"""
Technical problem: {description}
Context: {json.dumps(context)}

Relevant past experiences:
{memory_context}

Generate a solution or advice for this problem:
"""
        
        return self._generate_with_llm(prompt, max_tokens=300)

# Programmer Agent
class ProgrammerAgent(Agent):
    current_tasks: List[Task] = []
    completed_tasks: